# Предсказание стоимости жилья

Требуется обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Для оценки качества используются метрики RMSE, MAE, R2.

## Описание проекта

### Columns:

* **longitude** — широта;
* **latitude** — долгота;
* **housing_median_age** — медианный возраст жителей жилого массива;
* **total_rooms** — общее количество комнат в домах жилого массива;
* **total_bedrooms** — общее количество спален в домах жилого массива;
* **population** — количество человек, которые проживают в жилом массиве;
* **households** — количество домовладений в жилом массиве;
* **median_income** — медианный доход жителей жилого массива;
* **median_house_value** — медианная стоимость дома в жилом массиве;
* **ocean_proximity** — близость к океану.

### Задача:

Построить две модели линейной регресии на разных наборах данных.
1. Модель, использующая все данные из файла.
2. Модель, использующая только числовые переменные, исключая все категориальные. 

Сравнить результат работы **Линейной регрессии** на двух наборах данных по метрикам:
* **RMSE**
* **MAE**
* **R2**

## Импорт, инициализация, чтение файлов.

### Импорт

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import mean,col,split, col, regexp_extract, when, lit
from pyspark.sql.functions import isnan


from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.regression  import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import OneHotEncoder    

RANDOM_SEED = 1667


### Инициализация Spark сессии 

In [2]:
spark = SparkSession.builder \
                    .master("local") \
                    .appName("EDA California Housing") \
                    .getOrCreate()

### Чтение содержимого файла housing.csv

In [3]:
dt_housing = spark.read.csv('/datasets/housing.csv', \
                             header = True, \
                             inferSchema = True)

dt_housing.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



## EDA

### Первичный осмотр:

In [4]:
print(pd.DataFrame(dt_housing.dtypes, columns=['column', 'type']).head(10))

               column    type
0           longitude  double
1            latitude  double
2  housing_median_age  double
3         total_rooms  double
4      total_bedrooms  double
5          population  double
6          households  double
7       median_income  double
8  median_house_value  double
9     ocean_proximity  string


Категориальные значения представлены только одной переменной: ocean_proximity

In [5]:
dt_housing.show(10)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

In [7]:
dt_housing.select(['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms']).describe().show()

+-------+-------------------+-----------------+------------------+------------------+------------------+
|summary|          longitude|         latitude|housing_median_age|       total_rooms|    total_bedrooms|
+-------+-------------------+-----------------+------------------+------------------+------------------+
|  count|              20640|            20640|             20640|             20640|             20433|
|   mean|-119.56970445736148| 35.6318614341087|28.639486434108527|2635.7630813953488| 537.8705525375618|
| stddev|  2.003531723502584|2.135952397457101| 12.58555761211163|2181.6152515827944|421.38507007403115|
|    min|            -124.35|            32.54|               1.0|               2.0|               1.0|
|    max|            -114.31|            41.95|              52.0|           39320.0|            6445.0|
+-------+-------------------+-----------------+------------------+------------------+------------------+



In [8]:
dt_housing.select(['population',
 'households',
 'median_income',
 'median_house_value',
 'ocean_proximity']).describe().show()

+-------+------------------+-----------------+------------------+------------------+---------------+
|summary|        population|       households|     median_income|median_house_value|ocean_proximity|
+-------+------------------+-----------------+------------------+------------------+---------------+
|  count|             20640|            20640|             20640|             20640|          20640|
|   mean|1425.4767441860465|499.5396802325581|3.8706710029070246|206855.81690891474|           null|
| stddev|  1132.46212176534|382.3297528316098| 1.899821717945263|115395.61587441359|           null|
|    min|               3.0|              1.0|            0.4999|           14999.0|      <1H OCEAN|
|    max|           35682.0|           6082.0|           15.0001|          500001.0|     NEAR OCEAN|
+-------+------------------+-----------------+------------------+------------------+---------------+



Иследуем данные на наличие пропусков:

In [9]:
columns = dt_housing.columns


for column in columns:
    print(column, dt_housing.where(F.isnan(column) | F.col(column).isNull()).count())
    

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 207
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


~~Т.к. пропущенные значения не составляют значительный объем - удалим строки содержащие пропуски.~~

In [10]:
dt_housing = dt_housing.na.fill({"total_bedrooms" : int(dt_housing.select(F.mean("total_bedrooms")).collect()[0][0]) })

## Подготовка данных

### Разделение данных

In [11]:
cat_col = ['ocean_proximity']
num_col = ['longitude','latitude','housing_median_age','total_rooms','total_bedrooms','population','households','median_income']
target = 'median_house_value'

### Трансформация категориальных признаков

In [12]:
indexer = StringIndexer(inputCols = cat_col,
                       outputCols = [c+'_idx' for c in cat_col])

dt_housing = indexer.fit(dt_housing).transform(dt_housing)

cols = [c for c in dt_housing.columns for i in cat_col if (c.startswith(i))]

dt_housing.select(cols).show(3)

+---------------+-------------------+
|ocean_proximity|ocean_proximity_idx|
+---------------+-------------------+
|       NEAR BAY|                3.0|
|       NEAR BAY|                3.0|
|       NEAR BAY|                3.0|
+---------------+-------------------+
only showing top 3 rows



In [13]:
encoder = OneHotEncoder(inputCols = [c+'_idx' for c in cat_col],
                       outputCols = [c+'_ohe' for c in cat_col])

dt_housing = encoder.fit(dt_housing).transform(dt_housing)

cols = [c for c in dt_housing.columns for i in cat_col if (c.startswith(i))]

dt_housing.select(cols).show(3)

+---------------+-------------------+-------------------+
|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|
+---------------+-------------------+-------------------+
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
+---------------+-------------------+-------------------+
only showing top 3 rows



Объединяем признаки в один вектор, с которым алгоритм умеет работать.

In [14]:
categorical_assembler = \
                        VectorAssembler(inputCols = [c+'_ohe' for c in cat_col],
                                                       outputCol = 'categorial_features')

dt_housing = categorical_assembler.transform(dt_housing)

In [15]:
dt_housing.select('categorial_features').toPandas().head()

,categorial_features
0,"(0.0, 0.0, 0.0, 1.0)"
1,"(0.0, 0.0, 0.0, 1.0)"
2,"(0.0, 0.0, 0.0, 1.0)"
3,"(0.0, 0.0, 0.0, 1.0)"
4,"(0.0, 0.0, 0.0, 1.0)"


### Стандартизация числовых признаков

Произведем шкалирование значений, чтобы сильные выбросы не смещали предсказания модели.

In [16]:
numerical_assembler = VectorAssembler(inputCols=num_col, outputCol="numerical_features")
dt_housing = numerical_assembler.transform(dt_housing)


standardScaler = StandardScaler(inputCol='numerical_features',outputCol="numerical_features_scaled")
dt_housing = standardScaler.fit(dt_housing).transform(dt_housing) 

### Объединение категориальных и числовых значений

In [17]:
all_features = ['categorial_features','numerical_features_scaled']

final_assembler = VectorAssembler(inputCols = all_features,
                                  outputCol = "features")

dt_housing = final_assembler.transform(dt_housing)

dt_housing.select(all_features).show(3)

+-------------------+-------------------------+
|categorial_features|numerical_features_scaled|
+-------------------+-------------------------+
|      (4,[3],[1.0])|     [-61.007269596069...|
|      (4,[3],[1.0])|     [-61.002278409814...|
|      (4,[3],[1.0])|     [-61.012260782324...|
+-------------------+-------------------------+
only showing top 3 rows



### Разбиение на выборки 

In [18]:
train_dt, test_dt = dt_housing.randomSplit([.8,.2], seed = RANDOM_SEED)

print(train_dt.count(), test_dt.count())

16534 4106


## Обучение моделей

In [19]:
def lr(features):
    linreg = LinearRegression(labelCol=  target, featuresCol = features)
    model = linreg.fit(train_dt)
    return model.transform(test_dt)

In [20]:
predictions_all = lr('features')
predictions_num = lr('numerical_features_scaled')

23/07/27 15:31:03 WARN Instrumentation: [195d8b03] regParam is zero, which might cause numerical instability and overfitting.
23/07/27 15:31:04 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/07/27 15:31:04 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/07/27 15:31:04 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/07/27 15:31:04 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK
23/07/27 15:31:07 WARN Instrumentation: [0e29f303] regParam is zero, which might cause numerical instability and overfitting.


### ~~LinearRegression all features~~

~~lr_all = LinearRegression(labelCol=  target, featuresCol = 'features')~~

~~model = lr_all.fit(train_dt)~~

~~predictions_all = model.transform(test_dt)~~

### ~~LinearRegression numeral features~~

~~lr_num = LinearRegression(labelCol = target, featuresCol = 'numerical_features_scaled')~~

~~model_num = lr_num.fit(train_dt)~~

~~predictions_num = model_num.transform(test_dt)~~

## Анализ результатов

In [21]:
evl = RegressionEvaluator(predictionCol = 'prediction', labelCol = target)

### MAE

Mean Absolute Error 

Сумма абсолютных отклонений деленая на их количество. 


Представляет из себя разницу между предсказанием и фактическим значением целевого признака, деленого на количество объектов в выборке.


**Меньше - Лучше**

#### All features

In [22]:
evl.evaluate(predictions_all, {evl.metricName: "mae"})

50459.79702608826

#### Numeral features

In [23]:
evl.evaluate(predictions_num, {evl.metricName: "mae"})

51371.87883495836

### RMSE

Root Mean Squared Error

Корень из среднеквадратичной ошибки.

**Меньше - Лучше**

#### All features

In [24]:
evl.evaluate(predictions_all, {evl.metricName: "rmse"})

69091.58232451446

#### Numeral features

In [25]:
evl.evaluate(predictions_num, {evl.metricName: "rmse"})

69706.51542299568

### R2

Coefficient of determination


Вычисляет долю средней квадратичной ошибки модели от MSE среднего, а затем вычитает эту величину из единицы. Увеличение метрики означает прирост качества модели. 


**Больше - лучше**

#### All features

In [26]:
evl.evaluate(predictions_all, {evl.metricName: "r2"})

0.6491925920813164

#### Numeral features

In [27]:
evl.evaluate(predictions_num, {evl.metricName: "r2"})

0.642920248189606

In [28]:
spark.stop() 

## Итоговый вывод 

В рамках проекта были:
* Инициализирована локальная Spark-сессия
* Предобработаны данные
* Категориальные признаки закодированы как числовые
* Стандартезированы количественные признаки

* Используя метрики **RMSE**, **MAE** и **R2** было выполнено сравнение эффективности использования категориальных переменных в модели Линейной Регрессии, где для моделей использованы следующие наборы данных:
1. Весь набор переменных.
2. Набор использующий для обучения только числовые переменные, исключая категориальные.

Итого:

**На основе всех, трёх, метрик - наибольшую эффективность показывает модели использующая все доступные данные(т.е. категориальные и числовые переменные).**